# DAY 9 : SQL Analytics & Dashboards

In [0]:
%sql
-- Set the environment
USE CATALOG workspace;
CREATE SCHEMA IF NOT EXISTS sql_analytics_lab;
USE SCHEMA sql_analytics_lab;

-- Create Mock Sales Data with explicit Type Casting
CREATE OR REPLACE TABLE sales_data AS
SELECT 
    id as order_id,
    -- CAST the BIGINT to INT to resolve the [DATATYPE_MISMATCH]
    date_add(current_date(), CAST(-(id % 30) AS INT)) as order_date,
    (id * 123 % 500) as customer_id,
    CASE 
        WHEN id % 3 = 0 THEN 'Electronics'
        WHEN id % 3 = 1 THEN 'Fashion'
        ELSE 'Home & Garden'
    END as category,
    (id * 77 % 1000) as sale_amount,
    CASE 
        WHEN id % 10 < 7 THEN 'Completed'
        WHEN id % 10 < 9 THEN 'Pending'
        ELSE 'Cancelled'
    END as status
FROM range(1, 501); 

-- Quick check to ensure it worked
SELECT * FROM sales_data LIMIT 5;

order_id,order_date,customer_id,category,sale_amount,status
126,2026-01-11,498,Electronics,702,Completed
127,2026-01-10,121,Fashion,779,Pending
128,2026-01-09,244,Home & Garden,856,Pending
129,2026-01-08,367,Electronics,933,Cancelled
130,2026-01-07,490,Fashion,10,Completed


In [0]:
%sql
-- 1. Revenue Trends (Daily Aggregation)
-- We aggregate sale_amount FIRST, then apply the window function to those sums
SELECT 
    order_date, 
    SUM(sale_amount) as daily_revenue,
    AVG(SUM(sale_amount)) OVER(ORDER BY order_date ROWS BETWEEN 7 PRECEDING AND CURRENT ROW) as moving_avg_7d
FROM sales_data
WHERE status = 'Completed'
GROUP BY order_date
ORDER BY order_date;

-- 2. Top Performing Categories (Window Function)
SELECT 
    category,
    SUM(sale_amount) as total_revenue,
    RANK() OVER (ORDER BY SUM(sale_amount) DESC) as category_rank
FROM sales_data
GROUP BY category;

-- 3. Sales Funnel (Conversion Logic)
WITH funnel_base AS (
    SELECT 
        count(distinct customer_id) as total_users,
        count(distinct CASE WHEN status != 'Cancelled' THEN customer_id END) as engaged_users,
        count(distinct CASE WHEN status = 'Completed' THEN customer_id END) as converted_users
    FROM sales_data
)
SELECT 'Total Users' as stage, total_users as count FROM funnel_base
UNION ALL
SELECT 'Engaged Users', engaged_users FROM funnel_base
UNION ALL
SELECT 'Converted Users', converted_users FROM funnel_base;

stage,count
Total Users,500
Engaged Users,450
Converted Users,350
